In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
https://drive.google.com/drive/folders/1-_6K0X3q8BK-Urlu8na6AN15tzFsWzKR?usp=drive_link

In [2]:
data = pd.read_csv(r"C:\Users\basde\Downloads\titanic.csv", delimiter = ";")

In [3]:
data['Fare'] = data['Fare'].apply(lambda x: x.replace('.', '', x.count('.') - 1))

In [4]:
data['Fare'] = data['Fare'].astype('float')

In [5]:

data['Fare'].fillna(data['Fare'].mean(), inplace=True)

In [6]:
import re
data['Name'] = data['Name'].apply(lambda x: re.sub(r'[)]∗[^)]*', '', x).strip())
data['Name'] = data['Name'].str.replace(r'"[^"]*"', '', regex=True)
data['Name'] = data['Name'].str.split('(').str[0]
data['LastName'] = data['Name'].str.split(',').str.get(0).str.strip()
data['Name'] = data['Name'].str.split(',').str.get(1).str.strip()
data['Titel'] = data['Name'].str.split('.').str.get(0).str.strip()
data['Name'] = data['Name'].str.split('.').str.get(1).str.strip()
data['Titel'].replace({
    'Mlle':'Miss',
    'Ms':'Miss',
    'Mme':'Mrs',
    'Col':'Rare',
    'Major':'Rare',
    'Lady':'Rare',
    'Sir':'Rare',
    'Don':'Rare',
    'Capt':'Rare',
    'the Countess':'Rare',
    'Jonkheer':'Rare',
    'Dona':'Rare'
}, inplace=True)
data.drop('Name', axis=1, inplace=True)
data.drop('LastName', axis=1, inplace=True)

In [7]:
data['Fam_Size'] = data['SibSp'] + data['Parch'] + 1 
data['Has_Relatives'] = data.apply(lambda row: True if row['Parch'] > 0 or row['SibSp'] > 0 else False, axis=1)
data.drop('Parch', axis=1, inplace=True)
data.drop('SibSp', axis=1, inplace=True)

In [8]:
data.drop('Ticket', axis=1, inplace=True)

In [9]:
data['Deck'] = data['Cabin'].astype(str).str[0]

def cabin_estimator(i):
    """Grouping Deck feature by the first letter via Fare"""
    a = 0
    if i<16:
        a = "G"
    elif i>=16 and i<27:
        a = "F"
    elif i>=27 and i<38:
        a = "T"
    elif i>=38 and i<47:
        a = "A"
    elif i>= 47 and i<53:
        a = "E"
    elif i>= 53 and i<54:
        a = "D"
    elif i>=54 and i<116:
        a = 'C'
    else:
        a = "B"
    return a
with_N = data[data['Deck'] == "n"]
without_N = data[data['Deck'] != "n"]
with_N['Fare'] = pd.to_numeric(with_N['Fare'], errors='coerce')
with_N['Deck'] = with_N.Fare.apply(lambda x: cabin_estimator(x))
##applying cabin estimator function. 
concatenated_df = pd.concat([with_N, without_N], axis=0)
concatenated_df.sort_index(inplace=True)
## getting back Cabin. 
data['Deck'] = concatenated_df['Deck']
data['Deck']

0      G
1      C
2      G
3      C
4      G
      ..
886    G
887    B
888    F
889    C
890    G
Name: Deck, Length: 891, dtype: object

In [10]:
data['Fare'] = pd.qcut(data['Fare'], q=6, labels=['1','2','3','4','5','6'])

In [11]:
data['Deck'].value_counts()


Deck
G    298
C    219
B    128
F    112
T     40
E     35
D     33
A     26
Name: count, dtype: int64

In [12]:
data.drop('Embarked', axis=1, inplace=True)
data.drop('Cabin', axis=1, inplace=True)
data.drop('Fam_Size', axis=1, inplace=True)

In [13]:
data['Age'] = data.groupby(['Titel'])['Age'].transform(lambda x: x.fillna(x.median()))
# hier vul ik de missende leeftijden volgens het gemiddelde per titel

In [14]:
data['Age'] = pd.cut(data['Age'], bins=[0, 15, 29, 60, 100] , labels= [0, 1, 2, 3], right=True)

In [15]:
data['Has_Relatives'] = data['Has_Relatives'].replace({True: 1, False: 0})

In [21]:
le = LabelEncoder()
data['Titel'] = le.fit_transform(data['Titel'])
data['Deck'] = le.fit_transform(data['Deck'])

In [22]:
data

,PassengerId,Survived,Pclass,Sex,Age,Fare,Titel,Has_Relatives,Deck
0,1,0,3,1,1,1,3,1,6
1,2,1,1,0,2,6,4,1,2
2,3,1,3,0,1,1,2,0,6
3,4,1,1,0,2,4,4,1,2
4,5,0,3,1,2,1,3,0,6
...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,1,2,6,0,6
887,888,1,1,0,1,4,2,0,1
888,889,0,3,0,1,3,2,1,5
889,890,1,1,1,1,4,3,0,2
